
<a id='0'></a> 
# Описание проекта "Predict TripAdvisor Rating".

В рамках данного проекта требуется предсказать рейтинг ресторана в TripAdvisor. Работу над проектом будем проводить по следующим этапам:

1. Импорт библиотек, подготовка функций, чтение и первичный анализ данных:

<a href='#1.1'>1.1. Импорт библиотек, подготовка функций.</a> 

<a href='#1.2'>1.2. Чтение и первичный анализ данных.</a> 

2. Предобработка данных:

<a href='#2.1'>2.1. Обработка признака "cuisine_style".</a> 

<a href='#2.2'>2.2. Обработка признака "price_range".</a> 

<a href='#2.3'>2.3. Обработка признака "number_of_reviews".</a> 

3. Исследовательский анализ данных:

<a href='#3.1'>3.1. Признак "restaurant_id".</a> 

<a href='#3.2'>3.2. Признак "ranking".</a> 

<a href='#3.3'>3.3. Признак "city".</a> 

<a href='#3.4'>3.4. Признак "rating".</a> 

4. Создание новых признаков:

<a href='#4.1'>4.1. Признак "cuisine_style".</a> 

<a href='#4.2'>4.2. Признак "restaurant_id".</a> 

<a href='#4.3'>4.3. Признак "reviews".</a> 

<a href='#4.4'>4.4. Признак "city".</a> 

<a href='#4.5'>4.5. Признаки "ranking" и "number_of_reviews".</a> 

<a href='#4.6'>4.6. Отбор признаков.</a> 

5. Препроцессинг:

<a href='#5.1'>5.1. Написание функции для предобработки данных и создания новых признаков.</a> 

6. Обучение и тестирование модели:

<a href='#6.1'>6.1. Обучение и тестирование модели.</a> 

Метрикой качества модели будет средняя квадратическая ошибка (МАЕ).

# Этап 1. Импорт библиотек, подготовка функций, чтение и первичный анализ данных.

<a id='1.1'></a> 
## Этап 1.1. Импорт библиотек, подготовка функций.

Импортируем нужные для работы библиотеки.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import os
from datetime import date
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 999

Подготовим несколько функций, которые пригодятся при обработке данных.

In [ ]:
def df_info(df):
    print('Первые 10 строк набора данных')
    display(df.head(10))
    print()
    print('Информация о наборе данных:')
    print(df.info())
    print()
    print('Размер набор данных: {} признаков, {} объектов'.format(df.shape[1], df.shape[0]))
    print('Дубликаты:', df.duplicated().sum())
    print('Пропуски:')
    display(df.isna().sum())
    for i in df.columns:
        print(df[i].value_counts())
        
def date_processing(row):
    if row['dates_of_reviews_count'] == 1:
        row['first_review'] = pd.to_datetime(row['dates_of_reviews'][0])
        row['second_review'] = pd.to_datetime(row['dates_of_reviews'][0])
    
    elif row['dates_of_reviews_count'] == 0:
        #добавим 2 идентичные даты, чтобы признак не был пропущенным и чтобы можно было посчитать разницу для других признаков
        row['first_review'] = date(2010, 1, 1)
        row['second_review'] = date(2010, 1, 1)
    
    else:
        if pd.to_datetime(row['dates_of_reviews'][0]) < pd.to_datetime(row['dates_of_reviews'][1]):
            row['first_review'] = pd.to_datetime(row['dates_of_reviews'][1])
            row['second_review'] = pd.to_datetime(row['dates_of_reviews'][0])
        else:
            row['first_review'] = pd.to_datetime(row['dates_of_reviews'][0])
            row['second_review'] = pd.to_datetime(row['dates_of_reviews'][1])
    return row                      

С помощью системной библиотеки os получим директорию, хранящую файлы с данными.

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
RANDOM_SEED = 42
!pip freeze > requirements.txt

### <a href='#0'>К оглавлению.</a> 

<a id='1.2'></a> 
## Этап 1.2. Чтение и первичный анализ данных.

Считаем данные, используя ранее полученные пути к их местоположению.

In [ ]:
data_dir = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(data_dir + '/main_task.csv')
df_test = pd.read_csv(data_dir + 'kaggle_task.csv')
sample_submission = pd.read_csv(data_dir + '/sample_submission.csv')

Для корректной обработки данных соединим тренировочный и тестовый наборы данных в единый набор, при этом тренировочный набор отметим меткой 1, тестовый - меткой 0.

Также в тестовом наборе отсутствует целевая переменная Rating, поэтому в тетсовом наборе данных создадим такой признак и заполним его константой. После обработки данных мы его удалим.

In [ ]:
df_train['sample'] = 1
df_test['sample'] = 0
df_test['Rating'] = 0 
df = df_test.append(df_train, sort=False).reset_index(drop=True)

Применим ранее подготовленную функцию для проведения мини-EDA набора данных.

In [ ]:
df_info(df)

Описание признаков:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

Для удобства обращения к признакам сведём их к нижнему регистру, а также заменим пробелы на нижние подчёркивания.

In [ ]:
df.columns = [str(i).lower().replace(' ', '_') for i in df.columns]

# Вывод по этапу 1.

На данном этапе мы испортировали необходимые для работы библиотеки, подготовили несколько функций для анализа и обработки данных, а также провели мини-EDA набора данных.

### <a href='#0'>К оглавлению.</a> 

# Этап 2. Предобработка данных.

На данном этапе осуществим предварительную обработку данных.

<a id='2.1'></a> 
## Этап 2.1. Признак "cuisine_style".

В признаке cuisine_style представлена информация о кухнях, подаваемых в рестоане.

В данном признаке есть 2 глобальных затруднения:

1. Данные представлены в виде списка, но в форме строки. С такими данными довольно проблематично работать "из коробки".

2. В данных есть пропуски.

Первую проблему решать не будем: как фактор для обучения модели данный признак неинформативен, однако с ним ещё предстоит поработать на этапе создания новых признаков.

Пропущенные значения заменим на наиболее часто встречающуюся кухню в наборе данных.

Чтобы это сделать, сначала выделим из строки каждую отдельную кухню.

In [ ]:
top_cuisine = df[['cuisine_style']]
top_cuisine['cuisine_style'] = top_cuisine['cuisine_style'].str.split(',')

Далее с помощью функции explode развернём списки с наименованиями кухонь.

In [ ]:
top_cuisine = top_cuisine['cuisine_style'].explode()
top_cuisine

Видим, что список удалось успешно раскрыть, однако данные содержат много мусора. Их нужно очистить.

In [ ]:
top_cuisine = top_cuisine.apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", "").strip())
top_cuisine

Теперь выясним, какая кухня является наиболее часто встречаемой.

In [ ]:
top_cuisine = top_cuisine.value_counts(ascending=False).index[0]
top_cuisine

Видим, что кухня Vegetarian Friendly присутствует в большинстве исследуемых ресторанов. Заменим пропущенные значения признака cuisine_style на значение Vegetarian Friendly.

In [ ]:
df['cuisine_style'] = df['cuisine_style'].fillna(top_cuisine)

Посмотрим, что получилось.

In [ ]:
df.head()

In [ ]:
df['cuisine_style'].isna().sum()

Пропуски в признаке "cuisine_style" успешно обработаны.

### <a href='#0'>К оглавлению.</a> 

<a id='2.2'></a> 
## Этап 2.2. Признак "price_range".

В признаке price_range также присутствуют пропуски. Их - как и в случае с кухнями - заменим на значение наиболее часто встречающегося значения ценового сегмента ресторана.

In [ ]:
df['price_range'].value_counts(ascending=False)

Видим, что большинство ресторанов находятся в среднем ценовом сегменте. Заменим пропуски на это значение.

In [ ]:
df['price_range'] = df['price_range'].fillna('$$ - $$$')

Так как признак price_range является последовательным, т.е. каждое значение может быть сравнено с другими значениями, имеет смысл закодировать значения признака в числовое представление, с которым сможет работать алгоритм машинного обучения.

Для этого создадим соответствующий словарь, в котором ключ - ценовой сегмент ресторана в старом представлении, а значение - закодированное значение ценового сегмента.

In [ ]:
price_dict = {'$' : 1, '$$ - $$$' : 2, '$$$$' : 3}
df['price_range'] = df['price_range'].map(price_dict)

Проверим, что получилось.

In [ ]:
df.head()

Признак обработан успешно.

### <a href='#0'>К оглавлению.</a> 

<a id='2.3'></a> 
## Этап 2.3. Признак "number_of_reviews".

В признаке number_of_reviews также присутствуют пропуски.

В данном случае отсутствие значения в признаке, характеризующем количество ревью для определённого ресторана, резонно предположить, что отзывов у ресторана нет. Поэтому пропущенные значения заменяем на 0.

In [ ]:
df['number_of_reviews'] = df['number_of_reviews'].fillna(0)

Посмотрим, что получилось.

In [ ]:
df['number_of_reviews'].isna().sum()

Пропуски успешно обработаны.

### <a href='#0'>К оглавлению.</a> 

# Вывод по этапу 2.

На данном этапе мы осуществили предварительную обработку данных по следующим направлениям:

1. В признаке cuisine_style пропущенные значения заменили на наиболее часто встречающуюся в ресторанах кухню - Vegetarian Friendly.

2. В признаке price_range пропущенные значения заменили на наиболее часто встречающийся ценовой сегмент ресторана в наборе данных, а также закодировали категориальный признак в цифровое представление.

3. В признаке number_of_reviews пропущенные значения заменили на 0.

### <a href='#0'>К оглавлению.</a> 

# Этап 3. Исследовательский анализ данных.

На данном этапе проведём исследовательский анализ данных, чтобы выявить интересные закономерности в данных. 

<a id='3.1'></a> 
## Этап 3.1. Признак "restaurant_id".

Сам по себе признак является бесполезным и не будет участвовать в обучении модели, однако из этапа 1 было видно, что один и тот же ресторан встречается в наборе данных несколько раз.

In [ ]:
df['restaurant_id'].value_counts()

Из этого можно предположить, что в наборе данных есть рестораны, представленные в нескольких городах.

In [ ]:
df.groupby('restaurant_id')['city'].count()

Так оно и оказалось. Это может стать дополнительным признаком для модели: например, признак примет значение 1, если это сеть ресторанов, и 0, если ресторан только один.

### <a href='#0'>К оглавлению.</a> 

<a id='3.2'></a> 
## Этап 3.2. Признак "ranking".

Исследуем признак ranking.

In [ ]:
plt.figure(figsize=(20, 5))
df['ranking'].hist(bins=100);

Видим, что распределение данных имеет длинный правый хвост. Это говорит о существенном дисбалансе в значениях признака. Это может быть связано с тем, что в небольших городах присутствует малое количество ресторанов, что оказывает влияние на распределение этого признака. Проверим.

In [ ]:
plt.figure(figsize=(20, 5))
for city in (df['city'].value_counts())[:10].index:
    df['ranking'][df['city'] == city].hist(bins=100)
plt.show()

Видим, что распределение стало более однородным, однако границы распределения меняются в зависимости от города. Таким образом, подтверждаем гипотезу о том, что размер города влияет на распределение переменной ranking.

Есть две идеи о нивелировании этого воздействия.

Во-первых, этот признак стоит нормировать через z-преобразование.

Во-вторых, модель корректно учитывала данный признак для стран с разной величиной, его стоит забинить. Однако это осуществим на этапе создания новых признаков.

### <a href='#0'>К оглавлению.</a> 

<a id='3.3'></a> 
## Этап 3.3. Признак "city".

Исследуем признак city. Прежде всего интересует, как количество ресторанов отличается от города к городу.

In [ ]:
plt.figure(figsize=(20, 10))
df.groupby('city')['restaurant_id'].nunique().sort_values(ascending=True).plot(kind='bar')

Как видим, распределение ресторанов по городам также далеко от нормального. Это объясняет дисбаланс распределения данных в признаке ranking. 

В качестве дополнительных признаков можно сделать дамми-кодирование признака city.

### <a href='#0'>К оглавлению.</a> 

<a id='3.4'></a> 
## Этап 3.4. Признак "rating".

Рассмотрим переменную rating.

In [ ]:
plt.figure(figsize=(20, 5))
df['rating'].value_counts(ascending=False).plot(kind='barh');

Видим, что в наборе данных больше всего собрано информации о ресторанах с высокими оценками (от 3 и выше), и с оценками 0. Может быть, оценка 0 означает, что отзыв мог быть не проставлен. 

Меньше всего отзывов собрано для ресторанов с оценками > 0 и < 3. Вероятно, именно в этих местах модель будет ошибаться чаще.

### <a href='#0'>К оглавлению.</a> 

# Вывод по этапу 3.

На данном этапе мы провели исследовательский анализ данных некоторых имеющихся признаков.

Удалось выяснить, что:

1. В наборе данных встрачаются как одиночные рестораны, так и сети ресторанов.

2. Распределение признака Ranking сильно зависит от размера города и количества в нём ресторанов.

3. В разных городах имеется разное количество уникальных ресторанов.

4. В наборе данных больше всего информации об отзывах с оценкой от 3 и выше, а также с оценкой 0.

### <a href='#0'>К оглавлению.</a> 

# Этап 4. Создание новых признаков.

На данном этапе создадим новые признаки из имеющихся.

<a id='4.1'></a> 
## Этап 4.1. Признак "cuisine_style".

Поработаем с кухнями. 

Для начала создадим признак, который показывает, сколько разных кухонь представлено в ресторане. При этом пропущенные значения мы заменяли на значение Vegetarian Friendly, поэтому в случае наличия толькой такой кухни будем присваивать значение 1. В остальных случаях посчитаем количество кухонь в списке.

In [ ]:
df['cuisine_count'] = df['cuisine_style'].apply(lambda x: len(x.split(',')) if x != 'Vegetarian Friendly' else 1)

Также создадим признак, показывающий, есть ли среди кухонь ресторана наиболее популярная - Vegetarian Friendly: если есть, присвоим признаку значение 1, если нет - 0.

In [ ]:
df['have_top_cuisine'] = df['cuisine_style'].apply(lambda x: 1 if top_cuisine in x else 0)

Посмотрим, что получилось.

In [ ]:
df.head()

Новый признак успешно создан.

### <a href='#0'>К оглавлению.</a> 

<a id='4.2'></a> 
## Этап 4.2. Признак "restaurant_id".

Ранее мы заметили, что в наборе данных присутствуют как одиночные рестораны, так и сети ресторанов.

Создадим признак, который это покажет: если ресторан уникальный, присвоим значение 0, если сеть - значение 1.

Для начала узнаем, какие рестораны встречаются в наборе данных только 1 раз. Сформируем список таких ресторанов.

In [ ]:
unique_restaurants = df['restaurant_id'].value_counts()[df['restaurant_id'].value_counts() == 1].index.to_list()

Далее создадим новый признак: если ресторан присутствует в списке уникальных ресторанов, присваиваем признаку значение 0, в противном случае - значение 1.

In [ ]:
df['net_or_unique'] = df['restaurant_id'].apply(lambda x: 0 if x in unique_restaurants else 1)

Посмотрим, что получилось.

In [ ]:
df.head()

Новый признак успешно создан.

### <a href='#0'>К оглавлению.</a> 

<a id='4.3'></a> 
## Этап 4.3. Признак "reviews".

Пожалуй, самый необычный признак. Согласно описанию, данный признак содержит даты двух последних ревью. 

Извлечём эти даты с помощью регулярных выражений.

In [ ]:
pattern = re.compile("\d+\/\d+\/\d+")
df['dates_of_reviews'] = df['reviews'].apply(lambda x: pattern.findall(str(x)))

Проверим.

In [ ]:
df.head()

Мы получили список, содержащий даты последних ревью. Однако на текущий момент эти даты представлены в виде строки. Чтобы с ними было удобно работать, преобразуем их в формат datetime.

In [ ]:
df['dates_of_reviews'] = df['dates_of_reviews'].apply(lambda x: [pd.to_datetime(i).date() for i in x])

Модно заметить, что иногда в отзыве нет даты, иногда она одна. Стоит добавить признак, который покажет, сколько у нас имеется датированных отзывов.

In [ ]:
df['dates_of_reviews_count'] = df['dates_of_reviews'].apply(lambda x: len(x))

Посмотрим на распределение признака.

In [ ]:
df['dates_of_reviews_count'].value_counts()

Чаще всего встречается ситуация с двумя датированными отзывами. А вот наличие трёх датированных отзывов уже не стыкуется с описанием данных. Жить не мешает, но всё же посмотрим на них.

In [ ]:
df[df['dates_of_reviews_count'] == 3]

Как видим, третий отзыв всегда не является свежее двух предыдущих, что нам на руку.

Будет полезно создать признаки с первым и вторым отзывом, чтобы сравнить их между собой. При этом важно учесть, что порядок нахождения отзыва в списке не говорит о том, какой отзыв был оставлен раньше, а какой позже. Поэтому даты придётся ещё и сравнивать. Для этого воспользуемся подготовленной функцией date_processing. Она учитывает наличие и только одного отзыва, и отсутствие отзыва в принципе, и наличие трёх отзывов.

In [ ]:
df = df.apply(date_processing, axis=1)

Проверим, что получилось.

In [ ]:
df.head()

Даты успешно обработали. Теперь можно посчитать разницу в днях между первым и вторым отзывами.

In [ ]:
df['review_timedelta'] = (pd.to_datetime(df['first_review']) - pd.to_datetime(df['second_review'])).dt.days

Посмотрим, как распределён признак.

In [ ]:
df['review_timedelta'].value_counts().sort_values()

Видим, что в признаке есть как небольшие разбежки, так и неадекватно большие. Последние, вероятно, возникли из-за неправильного ввода данных. Удалять их необязательно, но и оставить тоже не хотелось бы. Решение - применить биннинг. Количество групп - 10 - было подобрано эмпирически.

In [ ]:
df['review_timedelta'] = pd.cut(df['review_timedelta'], 10)

Проверим, что получилось.

In [ ]:
df.head()

Биннинг провели, однако в таком виде их в модель не передашь. Нужно закодировать. Для этого создадим словарь и с его помощью осуществим кодировку.

In [ ]:
interval_dict = df['review_timedelta'].value_counts().to_dict()
interval_dict

Пока не очень похоже на то, что нужно: словарь есть, но его значения совсем не подходят под наши потребности. Немного исправим словарь с помощью цикла.

In [ ]:
for i, j in enumerate(interval_dict):
    interval_dict[j] = i
interval_dict

Гораздо лучше. Можно осуществлять кодировку.

In [ ]:
df['review_timedelta'] = df['review_timedelta'].map(interval_dict)

Также у нас появились признаки-даты, из которых до кучи можно извлечь пару-тройку признаков: день, месяц, год и день недели отзыва.

In [ ]:
df['day_first_review'] = df['first_review'].dt.day
df['month_first_review'] = df['first_review'].dt.month
df['year_first_review'] = df['first_review'].dt.year
df['day_of_week_first_review'] = df['first_review'].dt.dayofweek
df['day_second_review'] = df['second_review'].dt.day
df['month_second_review'] = df['second_review'].dt.month
df['year_second_review'] = df['second_review'].dt.year
df['day_of_week_second_review'] = df['second_review'].dt.dayofweek

Проверим, что получилось.

In [ ]:
df.head()

Видим, что мы успешно создали ряд новых признаков на основе дат первого и второго ревью.

### <a href='#0'>К оглавлению.</a> 

<a id='4.4'></a> 
## Этап 4.4. Признак "city".

Поработаем с признаком city. Ранее мы выяснили, что количество ресторанов в городе может быть различным. Сделаем для каждого города признак, описывающий количество уникальных ресторанов для этого города.

In [ ]:
restaurants_count_dict = df.groupby('city')['restaurant_id'].nunique().to_dict()
df['restaurants_in_city_count'] = df['city'].map(restaurants_count_dict)

Далее из данного признака сделаем дамми-переменные, при этом - чтобы не попасть в ловушку дамми-переменных из-за эффекта мультиколлинеарности - исключим из спектра дамми-переменных одно значение признака. То есть при n-городов у нас останется n-1 дамми-признак. Зная n-1 значений соответствующих дамми-признаков, без труда можно восстановить последний дамми-признак.

In [ ]:
df = pd.get_dummies(data=df, columns=['city'], drop_first=True)

Проверим.

In [ ]:
df.head()

Дамми-признаки на основе признака "city" успешно созданы.

### <a href='#0'>К оглавлению.</a> 

<a id='4.5'></a> 
## Этап 4.5. Признаки "ranking" и "number_of_reviews".

Поработаем с признаками ranking и number_of_reviews.

Ранее мы говорили о необходимости стандартизации значений признака ranking и применения к нему биннинга. Стандартизировать можно также и признак number_of_reviews.

Начнём с биннинга. Процедуру уже делали с признаком review_timedelta, делаем то же самое.

In [ ]:
df['ranking_bins'] = pd.cut(df['ranking'], 10)

Посмотрим, что получилось.

In [ ]:
df.head()

Далее кодируем значение нового признака.

In [ ]:
ranking_bins_dict = df['ranking_bins'].value_counts().to_dict()
ranking_bins_dict

In [ ]:
for i, j in enumerate(ranking_bins_dict):
    ranking_bins_dict[j] = i
ranking_bins_dict

Кодируем.

In [ ]:
df['ranking_bins'] = df['ranking_bins'].map(ranking_bins_dict)

Далее осуществим z-стандартизацию. Будем использовать соответствующий инструмент из библиотеки sklearn. Для чистоты эксперимента обучать инструмент будем только на обучающей выборке, и на её основе трансформировать тестовую выборку.

Для начала разделим их.

In [ ]:
train = df[df['sample'] == 1]
test = df[df['sample'] == 0]

Создадим объект инструмента шкалирования и обучим его на признаках "ranking" и "number_of_reviews" из обучающей выборки.

In [ ]:
scaler = StandardScaler()
scaler.fit(train[['ranking', 'number_of_reviews']])

Инструмент обучен. Применим его к данным - для обоих выборок, так как мы только обучили инструмент, но не преобразовывали обучающую выборку.

In [ ]:
train[['ranking', 'number_of_reviews']] = scaler.transform(train[['ranking', 'number_of_reviews']])
test[['ranking', 'number_of_reviews']] = scaler.transform(test[['ranking', 'number_of_reviews']])

Соединим наши выборки.

In [ ]:
df = test.append(train, sort=False).reset_index(drop=True) # объединяем

Посмотрим на результат.

In [ ]:
df.head()

Трансформация данных прошла хорошо.

### <a href='#0'>К оглавлению.</a> 

<a id='4.6'></a> 
## Этап 4.6. Отбор признаков.

Мы создали несколько новых признаков.

Избавимся от лишних признаков, которые не будут участвовать в обучении модели.

In [ ]:
df.drop(['restaurant_id', 'reviews', 'url_ta', 'id_ta', 'cuisine_style', 'dates_of_reviews', 'first_review', 
         'second_review'], axis=1, inplace=True)

Посмотрим на корреляционную матрицу и избавимся от скоррелированных признаков.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(df.drop(['sample'], axis=1).corr(),cmap="BrBG")
df.drop(['sample'], axis=1).corr()

Как видим, в наборе данных нет сильно скоррелированных с целевой переменной признаков, эффект мультиколлинеарности также не наблюдается.

### <a href='#0'>К оглавлению.</a> 

# Вывод по этапу 4.

На данном этапе мы создали новые признаки на основе уже имеющихся в наборе данных:

1. На основе признака "cuisine_style" мы создали признак, показывающий количество кухонь, представленных в ресторане, а также признак наличия в ресторане самой популярной кухни.

2. На основе признака "restaurant_id" мы создали признак, показывающий, чем является ресторан - уникальные заведением или сетью ресторанов.

3. На основе признака "reviews" мы создали два промежуточных параметра - дата первого и второго доступных ревью - и на их основе создали ряд новых признаков: разницу в днях между ревью, а также признаки-даты из этих двух дат: год, месяц, день и день недели первого и второго ревью.

4. С помощью признаков "city" и "restaurant_id" мы создали признак, описывающий количество уникальных ресторанов в каждом городе, а также дамми-переменные на основе признака "city", при этом для избежания эффекта мультиколлиреарности мы построили n-1 дамми-признак, где n - количество уникальных городов в наборе данных.

5. На основе признака "ranking" была создана переменная, разбивающая признак "ranking" на 10 групп по мере возрастания значения признака. Кроме того, признаки "ranking" и "number_of_reviews" были нормированы с помощью z-преобразования.

6. На последнем этапе мы избавились от ненужных для моделирования признаков, а также проверили признаки с помощью корреляционной матрицы на предмет сильной скоррелированности признаков с целеовой переменной и мультиколлинеарности.

### <a href='#0'>К оглавлению.</a> 

# Этап 5. Препроцессинг.

<a id='5.1'></a> 
## Этап 5.1. Написание функции для предобработки данных и создания новых признаков.

На данном этапе проделанные выше этапы предобработки данных и создания новых признаков обернём в единую функцию.

Для начала заново создадим исходный набор данных.

In [ ]:
data_dir = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(data_dir + '/main_task.csv')
df_test = pd.read_csv(data_dir + 'kaggle_task.csv')
sample_submission = pd.read_csv(data_dir + '/sample_submission.csv')

df_train['sample'] = 1
df_test['sample'] = 0
df_test['Rating'] = 0 
df = df_test.append(df_train, sort=False).reset_index(drop=True)

In [ ]:
def data_preprocessing(df):
    
    df.columns = [str(i).lower().replace(' ', '_') for i in df.columns]
    
    top_cuisine = df[['cuisine_style']]
    top_cuisine['cuisine_style'] = top_cuisine['cuisine_style'].str.split(',')
    top_cuisine = top_cuisine['cuisine_style'].explode()
    top_cuisine = top_cuisine.apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", "").strip())
    top_cuisine = top_cuisine.value_counts(ascending=False).index[0]
    df['cuisine_style'] = df['cuisine_style'].fillna(top_cuisine)
    
    df['price_range'] = df['price_range'].fillna('$$ - $$$')
    price_dict = {'$' : 1, '$$ - $$$' : 2, '$$$$' : 3}
    df['price_range'] = df['price_range'].map(price_dict)
    
    df['number_of_reviews'] = df['number_of_reviews'].fillna(0)
    
    df['cuisine_count'] = df['cuisine_style'].apply(lambda x: len(x.split(',')) if x != 'Vegetarian Friendly' else 1)
    
    df['have_top_cuisine'] = df['cuisine_style'].apply(lambda x: 1 if top_cuisine in x else 0)
    
    unique_restaurants = df['restaurant_id'].value_counts()[df['restaurant_id'].value_counts() == 1].index.to_list()
    df['net_or_unique'] = df['restaurant_id'].apply(lambda x: 0 if x in unique_restaurants else 1)
    
    pattern = re.compile("\d+\/\d+\/\d+")
    df['dates_of_reviews'] = df['reviews'].apply(lambda x: pattern.findall(str(x)))
    df['dates_of_reviews'] = df['dates_of_reviews'].apply(lambda x: [pd.to_datetime(i).date() for i in x])
    df['dates_of_reviews_count'] = df['dates_of_reviews'].apply(lambda x: len(x))
    df = df.apply(date_processing, axis=1)
    df['review_timedelta'] = (pd.to_datetime(df['first_review']) - pd.to_datetime(df['second_review'])).dt.days
    df['review_timedelta'] = pd.cut(df['review_timedelta'], 10)
    interval_dict = df['review_timedelta'].value_counts().to_dict()
    for i, j in enumerate(interval_dict):
        interval_dict[j] = i
    df['review_timedelta'] = df['review_timedelta'].map(interval_dict)
    df['day_first_review'] = df['first_review'].dt.day
    df['month_first_review'] = df['first_review'].dt.month
    df['year_first_review'] = df['first_review'].dt.year
    df['day_of_week_first_review'] = df['first_review'].dt.dayofweek
    df['day_second_review'] = df['second_review'].dt.day
    df['month_second_review'] = df['second_review'].dt.month
    df['year_second_review'] = df['second_review'].dt.year
    df['day_of_week_second_review'] = df['second_review'].dt.dayofweek
    
    restaurants_count_dict = df.groupby('city')['restaurant_id'].nunique().to_dict()
    df['restaurants_in_city_count'] = df['city'].map(restaurants_count_dict)
    
    df = pd.get_dummies(data=df, columns=['city'], drop_first=True)
    df['ranking_bins'] = pd.cut(df['ranking'], 10)
    
    ranking_bins_dict = df['ranking_bins'].value_counts().to_dict()
    for i, j in enumerate(ranking_bins_dict):
        ranking_bins_dict[j] = i
    df['ranking_bins'] = df['ranking_bins'].map(ranking_bins_dict)
    
    train = df[df['sample'] == 1]
    test = df[df['sample'] == 0]
    
    scaler = StandardScaler()
    scaler.fit(train[['ranking', 'number_of_reviews']])
    train[['ranking', 'number_of_reviews']] = scaler.transform(train[['ranking', 'number_of_reviews']])
    test[['ranking', 'number_of_reviews']] = scaler.transform(test[['ranking', 'number_of_reviews']])
    
    df = test.append(train, sort=False).reset_index(drop=True)
    df.drop(['restaurant_id', 'reviews', 'url_ta', 'id_ta', 'cuisine_style', 'dates_of_reviews', 'first_review', 
             'second_review'], axis=1, inplace=True)
    
    return df

Функция написана. Применим её к нашему набору данных.

In [ ]:
df = data_preprocessing(df)

Посмотрим, что получилось.

In [ ]:
df.head()

Данные в порядке.

### <a href='#0'>К оглавлению.</a> 

# Вывод по этапу 5.

На данном этапе мы собрали весь пайплайн предобработки данных и создания новых признаков в единую функцию. Это позволит быстро готовить аналогичные данные к повторному моделированию.

### <a href='#0'>К оглавлению.</a> 

# Этап 6. Обучение и тестирование модели.

<a id='6.1'></a> 
## Этап 6.1. Обучение и тестирование модели.

На данном этапе на ранее подготовленных данных обучим модель случайного леса и протестируем качество модели на метрике MAE.

Для начала разделим данные на тренировочный и тестовый наборы по признаку "sample", который ранее мы создавали именно для этих целей.

Признаки "sample" и "rating" удаляем из тестового набора данных, так как мы вводили их искусственно для упрощения процесса обработки данных.

Из обучающео набора данных удаляем только признак "sample", так как признак "rating" является целевым.

In [ ]:
test = df[df['sample'] == 0]
train = df[df['sample'] == 1]
train.drop('sample', axis=1, inplace=True)
test.drop(['rating', 'sample'], axis=1, inplace=True)

Перед отправокй ответов на Kaggle протестируем работу модели на обучающей выборке. Для этого обучающую выборку разделим на обучающую и валидационную. 

Сначала выделим признаки для обучения и целевой признак в отдельные наборы данных.

In [ ]:
features = train.drop(['rating'], axis=1)
target = train['rating']

Далее обучающую выборку разделим на обучающую и валидационную в соотношении 80/20 соответственно.

In [ ]:
features_train, features_validation, target_train, target_validation = train_test_split(features, target, test_size=0.20, random_state=RANDOM_SEED)

Приступим к работе с моделью. 

Обучим модель на обучающей выборке, протестируем её работу на валидационной выборке и сравним ответы модели с истинными значениями.

In [ ]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
model.fit(features_train, target_train)
predictions = model.predict(features_validation)
print('MAE:', round(metrics.mean_absolute_error(target_validation, predictions), 4))

Нам удалось добиться неплохого результата.

Посмотрим на значимость признаков с точки зрения модели.

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=features_train.columns)
feat_importances.nlargest(15).plot(kind='barh');

Наиболее значимым признаком в модели оказался ranking, следом за ним идут количество уникальных ресторанов в городе и количество ревью. Остальные признаки имеют относительно невысокую значимость для модели.

Теперь для обучения модели используем всю обучающую выборку, протестируем её работу на тестовой выборке и отправим ответы на Kaggle.

Для начала проверим имеющиеся выборки.

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
sample_submission.head(10)

Данные в порядке. Разделим тренировочную выборку на признаки для обучения и целевой признак.

In [ ]:
features_train = train.drop('rating', axis=1)
target_train = train['rating']

Обучим модель.

In [ ]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
model.fit(features_train, target_train)
predictions = model.predict(test)

Посмотрим на получившиеся предикты.

In [ ]:
predictions

Заменим полученными предиктами значения рейтинга в наборе данных sample_submission.

In [ ]:
sample_submission['Rating'] = predictions

Посмотрим, что получилось.

In [ ]:
sample_submission.head()

Выглядит как то, что нужно. Можно отправлять ответы на Kaggle.

In [ ]:
sample_submission.to_csv('submission_1.csv', index=False)

# Вывод по этапу 6.

На данном этапе мы обучили модель и протестировали её работу с помощью метрики MAE: удалось достигнуть значения метрики в 0.2076 на валидационной выборке.

### <a href='#0'>К оглавлению.</a> 

# Итоговый вывод по проекту.

В рамках данного проекта требовалось предсказать рейтинг ресторана в TripAdvisor. Работу над проектом проводили по следующим этапам:

1. Импорт библиотек, подготовка функций, чтение и первичный анализ данных.

2. Предобработка данных.

3. Исследовательский анализ данных.

4. Создание новых признаков.

5. Препроцессинг.

6. Обучение и тестирование модели.

Метрикой качества модели была средняя квадратическая ошибка (МАЕ).

Отметим некоторые выводы по каждому этапу работы.

## Вывод по этапу 1.

На данном этапе мы испортировали необходимые для работы библиотеки, подготовили несколько функций для анализа и обработки данных, а также провели мини-EDA набора данных.

## Вывод по этапу 2.

На данном этапе мы осуществили предварительную обработку данных по следующим направлениям:

1. В признаке cuisine_style пропущенные значения заменили на наиболее часто встречающуюся в ресторанах кухню - Vegetarian Friendly.

2. В признаке price_range пропущенные значения заменили на наиболее часто встречающийся ценовой сегмент ресторана в наборе данных, а также закодировали категориальный признак в цифровое представление.

3. В признаке number_of_reviews пропущенные значения заменили на 0.

## Вывод по этапу 3.

На данном этапе мы провели исследовательский анализ данных некоторых имеющихся признаков.

Удалось выяснить, что:

1. В наборе данных встрачаются как одиночные рестораны, так и сети ресторанов.

2. Распределение признака Ranking сильно зависит от размера города и количества в нём ресторанов.

3. В разных городах имеется разное количество уникальных ресторанов.

4. В наборе данных больше всего информации об отзывах с оценкой от 3 и выше, а также с оценкой 0.

## Вывод по этапу 4.

На данном этапе мы создали новые признаки на основе уже имеющихся в наборе данных:

1. На основе признака "cuisine_style" мы создали признак, показывающий количество кухонь, представленных в ресторане, а также признак наличия в ресторане самой популярной кухни.

2. На основе признака "restaurant_id" мы создали признак, показывающий, чем является ресторан - уникальные заведением или сетью ресторанов.

3. На основе признака "reviews" мы создали два промежуточных параметра - дата первого и второго доступных ревью - и на их основе создали ряд новых признаков: разницу в днях между ревью, а также признаки-даты из этих двух дат: год, месяц, день и день недели первого и второго ревью.

4. С помощью признаков "city" и "restaurant_id" мы создали признак, описывающий количество уникальных ресторанов в каждом городе, а также дамми-переменные на основе признака "city", при этом для избежания эффекта мультиколлиреарности мы построили n-1 дамми-признак, где n - количество уникальных городов в наборе данных.

5. На основе признака "ranking" была создана переменная, разбивающая признак "ranking" на 10 групп по мере возрастания значения признака. Кроме того, признаки "ranking" и "number_of_reviews" были нормированы с помощью z-преобразования.

6. На последнем этапе мы избавились от ненужных для моделирования признаков, а также проверили признаки с помощью корреляционной матрицы на предмет сильной скоррелированности признаков с целеовой переменной и мультиколлинеарности.

## Вывод по этапу 5.

На данном этапе мы собрали весь пайплайн предобработки данных и создания новых признаков в единую функцию. Это позволит быстро готовить аналогичные данные к повторному моделированию.

## Вывод по этапу 6.

На данном этапе мы обучили модель и протестировали её работу с помощью метрики MAE: удалось достигнуть значения метрики в 0.2076 на валидационной выборке.

### <a href='#0'>К оглавлению.</a> 